In [0]:
Create or Refresh Streaming Live Table Bronze_Table
Comment "This contains raw books data"
as Select * from cloud_files('dbfs:/mnt/demo-datasets/bookstore/books-cdc','json') ;

In [0]:
Create or Refresh Streaming Live Table Silver_Table;

APPLY CHANGES INTO LIVE.Silver_Table
  FROM STREAM(LIVE.Bronze_Table)
  KEYS (book_id)
  APPLY AS DELETE WHEN ROW_STATUS = "DELETE"
  SEQUENCE BY row_time
  COLUMNS * EXCEPT (ROW_STATUS, ROW_TIME);

In [0]:
CREATE OR REFRESH LIVE TABLE Gold_Table
COMMENT "Author books count"
AS SELECT author, count(*) as Author_Books_Count, current_timestamp() as Updated_time
from LIVE.Silver_Table
Group By author;

In [0]:
Create Live View books_sales as 
Select b.title, o.book.quantity from (
  select *, explode(books) as book
  from Live.orders_cleaned) as o
  inner join Live.Silver_Table as b
  on b.book_id = o.book.book_id;